In [1]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['NBA']

db.list_collection_names()

['historical_raptor',
 'player2k',
 'shot_chart',
 'game_log',
 'historical_pbp_modelled',
 'historical_pbp']

In [2]:
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import playbyplayv2, leaguegamelog, leaguegamefinder
from preprocess import feature_engineer
from nba_helper import read_url_to_csv, elo_url
from time import sleep

In [ ]:
games_to_update_df[['Home', 'Away']] = games_to_update_df['MATCHUP'].str.split('vs.', expand=True)
games_to_update_df.loc[:, 'Home'] = games_to_update_df['Home'].str.strip()
games_to_update_df.loc[:, 'home_team_win'] = games_to_update_df['WL'].replace({'W':1, 'L':0})

In [148]:
game_log = pd.DataFrame.from_records(db.game_log.find())

In [152]:
game_log.dropna(subset=['home_team_win'])

,_id,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,...,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Home,Away,home_team_win
0,61e9eb6cf0ee3d9953abc9ca,22015,1610612744,GSW,Golden State Warriors,21500003,2015-10-27,GSW vs. NOP,W,240,...,8,7,20,29,111,16,1,GSW,NOP,1.0
1,61e9eb6cf0ee3d9953abc9cb,22015,1610612741,CHI,Chicago Bulls,21500002,2015-10-27,CHI vs. CLE,W,240,...,6,10,13,22,97,2,1,CHI,CLE,1.0
2,61e9eb6cf0ee3d9953abc9cc,22015,1610612737,ATL,Atlanta Hawks,21500001,2015-10-27,ATL vs. DET,L,240,...,9,4,15,25,94,-12,1,ATL,DET,0.0
3,61e9eb6cf0ee3d9953abc9cd,22015,1610612747,LAL,Los Angeles Lakers,21500017,2015-10-28,LAL vs. MIN,L,240,...,2,4,14,29,111,-1,1,LAL,MIN,0.0
4,61e9eb6cf0ee3d9953abc9ce,22015,1610612756,PHX,Phoenix Suns,21500014,2015-10-28,PHX vs. DAL,L,240,...,3,3,18,30,95,-16,1,PHX,DAL,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7726,61ea18649ec8be579a2f50c6,22021,1610612762,UTA,Utah Jazz,22100677,2022-01-19,UTA vs. HOU,L,240,...,7,5,9,16,111,-5,1,UTA,HOU,0.0
7727,61ea18649ec8be579a2f50c7,22021,1610612758,SAC,Sacramento Kings,22100679,2022-01-19,SAC vs. DET,L,240,...,4,8,7,17,131,-2,1,SAC,DET,0.0
7728,61ea18649ec8be579a2f50c8,22021,1610612743,DEN,Denver Nuggets,22100678,2022-01-19,DEN vs. LAC,W,265,...,9,3,12,15,130,2,1,DEN,LAC,1.0
7729,61ea18649ec8be579a2f50c9,22021,1610612747,LAL,Los Angeles Lakers,22100680,2022-01-19,LAL vs. IND,L,240,...,4,7,11,18,104,-7,1,LAL,IND,0.0


In [141]:
game_log = game_log.drop(['Home', 'Away', 'home_team_win'], axis = 1)

In [142]:
game_log[['Home', 'Away']] = game_log['MATCHUP'].str.split('vs.', expand=True)
game_log.loc[:, 'Home'] = game_log['Home'].str.strip()
game_log.loc[:, 'home_team_win'] = game_log['WL'].replace({'W':1, 'L':0})

In [143]:
db.game_log.drop()

In [144]:
db.game_log.insert_many(game_log.to_dict('records'))


In [146]:
from pymongo import ASCENDING

In [147]:
db.game_log.create_index([("GAME_ID", ASCENDING)])

'GAME_ID_1'

In [112]:
games_to_delete_and_re_download = game_log.GAME_ID.unique()[-200:]

In [113]:
db.historical_pbp.delete_many({'GAME_ID': {'$in':games_to_delete_and_re_download.tolist()}})

In [114]:
db.historical_pbp_modelled.delete_many({'GAME_ID': {'$in':games_to_delete_and_re_download.tolist()}})

In [118]:
historical_pbp = pd.DataFrame.from_records(db.historical_pbp.find())

In [119]:
historical_pbp_modelled = pd.DataFrame.from_records(db.historical_pbp_modelled.find())

In [123]:
historical_pbp.GAME_ID.nunique()

7533

In [124]:
historical_pbp_modelled.GAME_ID.nunique()

7533

In [125]:
games_to_update = set(game_log.GAME_ID).difference(set(historical_pbp.GAME_ID.unique()))

In [126]:
len(games_to_update)

200

In [127]:
games_to_update_df = game_log[game_log.GAME_ID.isin(games_to_update)]

games_to_update_df[['Home', 'Away']] = games_to_update_df['MATCHUP'].str.split('vs.', expand=True)
games_to_update_df.loc[:, 'Home'] = games_to_update_df['Home'].str.strip()
games_to_update_df.loc[:, 'home_team_win'] = games_to_update_df['WL'].replace({'W':1, 'L':0})

/Users/arun/Projects/NBA/venv/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/arun/Projects/NBA/venv/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [128]:
elo = read_url_to_csv(elo_url)
elo = elo[elo['date'] > '2012-01-01']

elo.loc[:, 'elo_difference'] = np.abs(elo['elo1_pre'] - elo['elo2_pre'])

elo = elo[['date', 'team1', 'elo1_pre', 'elo2_pre', 'elo_difference']]

elo['team1'] = elo['team1'].replace({'BRK':'BKN',
                                    'PHO':'PHX',
                                    'CHO':'CHA',})

df = games_to_update_df.merge(elo, 
                     left_on=['GAME_DATE', 'Home'], 
                     right_on = ['date', 'team1'])[['GAME_ID', 'elo1_pre', 'elo2_pre', 'home_team_win', 'MATCHUP']]

In [129]:
pbps = []
for g_id in df['GAME_ID'].unique():
    pbps.append(playbyplayv2.PlayByPlayV2(f'00{g_id}').get_data_frames()[0])
    sleep(0.5)

In [132]:
pbp_df = pd.concat(pbps).astype({'GAME_ID':int})
pbp_df_prepped = pbp_df.groupby('GAME_ID').apply(feature_engineer).reset_index(drop=True)

final_df = pbp_df_prepped.merge(df, on=['GAME_ID'])

In [133]:
import tensorflow as tf

In [136]:
wEloCols = ['home_poss', 'diff', 'time_remaining', 'OT_ind', 'elo1_pre', 'elo2_pre']
wOEloCols = ['home_poss', 'diff', 'time_remaining', 'OT_ind']

model = tf.keras.models.load_model('../Models/TF_model_w_elo.h5')
model_wO_elo = tf.keras.models.load_model('../Models/TF_model_wO_elo.h5')

final_df.loc[:, 'preds_w_elo'] = model.predict_on_batch(final_df.fillna(0)[wEloCols])
final_df.loc[:, 'preds_wO_elo'] = model_wO_elo.predict_on_batch(final_df.fillna(0)[wOEloCols])

In [137]:
db.historical_pbp_modelled.insert_many(final_df.to_dict('records'))

In [138]:
db.historical_pbp.insert_many(pbp_df.to_dict('records'))